In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from utils.io_func import (
    load_from_pkl, load_from_npy, save_to_npy, load_from_pth,
)
from utils.helper import LSTMHelper
from config import SEED

In [ ]:
helper =LSTMHelper()

In [ ]:
BASE_SITE = "Site_1"
TEST_YEARS = [str(year) for year in [2018]]
X_PATH_TEMPLATE = "../preprocessing/out/{site}/x-corn_soybean-{year}.npy"
Y_PATH_TEMPLATE = "../preprocessing/out/{site}/y-corn_soybean-{year}.npy"
SCALER_PATH_TEMPLATE = (
    "../experiments/out/end_of_the_season/"
    "AtLSTM-corn_soybean/{site}/scaler.pkl"
)
MODEL_PATH_TEMPLATE = (
    "../experiments/out/end_of_the_season/"
    "AtLSTM-corn_soybean/{site}/atlstm.pth"
)
TSNE_RESULT_INPUT_FEATURE_PATH = (
    "./store/feature_vis_end2end/atlstm/tsne_result_input_feature.npy"
)
TSNE_RESULT_HIDDEN_FEATURE_PATH = (
    "./store/feature_vis_end2end/atlstm/tsne_result_hidden_feature.npy"
)
CLASS_NAME_SAMPLE_PATH = (
    "./store/feature_vis_end2end/atlstm/class_name_sample.npy"
)
DEVICE = torch.device("cuda:0")

# Input

In [ ]:
def get_paths(path_template, site, years):
    paths = []
    for year in years:
        paths.append(path_template.format(site=site, year=year))
    return paths


x_test = helper.input_x(get_paths(X_PATH_TEMPLATE, BASE_SITE, TEST_YEARS))
y_test = helper.input_y(get_paths(Y_PATH_TEMPLATE, BASE_SITE, TEST_YEARS))

In [ ]:
x_list, class_name_list = [], []
class_names = ["corn", "soybean"]
translator = {"corn": 0, "soybean": 1}
class_sample_size = 8000
for class_name in class_names:
    x_class = x_test[y_test==translator[class_name]]
    np.random.seed(SEED)
    class_sample_index = np.random.choice(
        x_class.shape[0], size=class_sample_size, replace=False
    )
    x_class_sample = x_class[class_sample_index]
    x_list.append(x_class_sample)
    class_name_list.append(np.full((class_sample_size,), class_name))
x_sample = np.concatenate(x_list)
class_name_sample = np.concatenate(class_name_list)
save_to_npy(class_name_sample, CLASS_NAME_SAMPLE_PATH)

In [ ]:
input_feature = x_sample.reshape(x_sample.shape[0], -1)  # for t-SNE

# Normalization

In [ ]:
scaler = load_from_pkl(SCALER_PATH_TEMPLATE.format(site=BASE_SITE))
x_sample = helper.normalize_with_scaler(scaler, x_sample)

# Hidden feature extraction

In [ ]:
net = helper.build_model()
net.load_state_dict(load_from_pth(MODEL_PATH_TEMPLATE.format(site=BASE_SITE)))
# store hidden features obtained from multiple GPUs in a list
# even if there is one GPU, the list should be used to avoid gc
hidden_feature_list = []


def store_hidden_features(self, input, output):
    hidden_feature_per_device = input[0].detach().squeeze()
    hidden_feature_list.append(hidden_feature_per_device)


net.fc.register_forward_hook(store_hidden_features)
net = nn.DataParallel(net, device_ids=[0, 1, 2, 3])
net.to(DEVICE)

In [ ]:
net.eval()
with torch.no_grad():
    net(torch.FloatTensor(x_sample).to(DEVICE))

In [ ]:
# use gpu label to reconstruct the order of hidden features
# ref: https://discuss.pytorch.org/t/register-forward-hook-with-multiple-gpus/12115
hidden_feature_list = sorted(
    hidden_feature_list, key=lambda x: int(str(x.device).split(":")[-1])
)

hidden_feature_list = [
    hidden_feature_per_device.cpu()
    for hidden_feature_per_device in hidden_feature_list
]
hidden_feature = np.concatenate(hidden_feature_list)

# t-SNE

In [ ]:
def project_to_2d(feature):
    transformer = PCA(n_components=50)
    transformed = transformer.fit_transform(feature)
    tsne = TSNE(n_components=2, verbose=1, perplexity=1500)
    return tsne.fit_transform(transformed)


tsne_result_input_feature = project_to_2d(input_feature)
save_to_npy(tsne_result_input_feature, TSNE_RESULT_INPUT_FEATURE_PATH)
tsne_result_hidden_feature = project_to_2d(hidden_feature)
save_to_npy(tsne_result_hidden_feature, TSNE_RESULT_HIDDEN_FEATURE_PATH)

In [ ]:
tsne_result_input_feature = load_from_npy(TSNE_RESULT_INPUT_FEATURE_PATH)
tsne_result_hidden_feature = load_from_npy(TSNE_RESULT_HIDDEN_FEATURE_PATH)
class_name_sample = load_from_npy(CLASS_NAME_SAMPLE_PATH)
plt.figure(figsize=(13, 5.5))


def scatter_plot(tsne_result_feature):
    sns.scatterplot(
        x=tsne_result_feature[:, 0], y=tsne_result_feature[:, 1],
        hue=class_name_sample,
        s=3.6, edgecolor="none"
    )
    plt.xticks([])
    plt.yticks([])
    plt.legend().set_visible(False)
    

plt.subplot(121)
scatter_plot(tsne_result_input_feature)
plt.title("Input features")
plt.subplot(122)
scatter_plot(tsne_result_hidden_feature)
plt.title("Learned features of AtLSTM")
plt.legend(
    loc='upper center', bbox_to_anchor=(-0.1, -0.08), ncol=3,
    handletextpad=0, columnspacing=2, frameon=False
)